In [2]:
import numpy as np
import librosa
from joblib import load

# Preprocessing and feature extraction functions
def preprocess_audio(file_path, target_sr=22050, noise_duration=0.5):
    from noisereduce import reduce_noise
    from scipy.signal import butter, filtfilt

    def ml_noise_reduction(y, sr, noise_duration):
        noise_sample = y[:int(noise_duration * sr)]
        y_denoised = reduce_noise(y=y, sr=sr, y_noise=noise_sample)
        return y_denoised

    def bandpass_filter(y, sr, low_cut=20, high_cut=500):
        nyquist = 0.5 * sr
        low = low_cut / nyquist
        high = high_cut / nyquist
        b, a = butter(1, [low, high], btype='band')
        return filtfilt(b, a, y)

    # Load audio
    y, sr = librosa.load(file_path, sr=target_sr)
    # Noise reduction
    y = ml_noise_reduction(y, sr, noise_duration)
    # Bandpass filtering
    y = bandpass_filter(y, sr)
    # Trim silent edges
    y, _ = librosa.effects.trim(y, top_db=20)
    # Normalize
    y = y / np.max(np.abs(y)) if np.max(np.abs(y)) != 0 else y
    return y, sr

def extract_features(signal, sr):
    mfccs = librosa.feature.mfcc(y=signal, sr=sr, n_mfcc=20).mean(axis=1)
    chroma = librosa.feature.chroma_stft(y=signal, sr=sr).mean(axis=1)
    spectral_contrast = librosa.feature.spectral_contrast(y=signal, sr=sr).mean(axis=1)
    zcr = librosa.feature.zero_crossing_rate(y=signal).mean(axis=1)
    rolloff = librosa.feature.spectral_rolloff(y=signal, sr=sr).mean(axis=1)
    centroid = librosa.feature.spectral_centroid(y=signal, sr=sr).mean(axis=1)
    rms_energy = librosa.feature.rms(y=signal).mean(axis=1)
    return {
        'mfccs': mfccs,
        'chroma': chroma,
        'spectral_contrast': spectral_contrast,
        'zcr': zcr,
        'rolloff': rolloff,
        'centroid': centroid,
        'rms_energy': rms_energy
    }

# Testing a new audio file
def predict_audio(file_path, model_path):
    # Step 1: Load the trained model, best feature combination, and labels
    model, best_features, labels = load(model_path)

    # Step 2: Preprocess the new audio file
    y, sr = preprocess_audio(file_path)
    
    # Step 3: Extract features from the new audio
    features = extract_features(y, sr)
    
    # Step 4: Combine the best feature combination
    X_new = np.concatenate([features[feat] for feat in best_features])
    X_new = X_new.reshape(1, -1)  # Reshape to 2D array for model input

    # Step 5: Predict the label
    predicted_label = model.predict(X_new)

    # Step 6: Map the numeric label back to the category
    label_mapping = {index: label for index, label in enumerate(labels)}
    predicted_category = label_mapping[predicted_label[0]]

    return predicted_category

# Define parameters
model_path = r"E:\OneDrive\Desktop\GP\Grad_Pro\models\Logistic Regression_best_model_with_features_and_labels.joblib"  # Path to the saved model
new_wav_path = r"(ORIGINAL)DATASET-HEARTSOUNDS/abnormal/a0002.wav"  # Path to the new audio file

# Predict category
predicted_category = predict_audio(new_wav_path, model_path)

# Output the result
print(f"The predicted category for the audio file is: {predicted_category}") 

c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The predicted category for the audio file is: abnormal
